In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 
import math
import utils.prepareDataCSV as prepare
import utils.baselineModels as baseline
import utils.train as train
import utils.plot as p
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn.functional as F
from torchinfo import summary
import torch.nn as nn
%matplotlib inline

In [6]:
train7path = '../../data/processed/stock0/newY_train7_part0.csv'
df = pd.read_csv(train7path)

In [7]:
ask1,ask1_volumn,bid1,bid1_volumn = np.array(df['0']),np.array(df['1']),np.array(df['2']),np.array(df['3'])

## 添加VOI特征

In [8]:
def sigma_Vb(Pb_now, Pb_past, Vb_now, Vb_past):
    if Pb_now<Pb_past:
        return 0
    elif Pb_now>Pb_past:
        return Vb_now
    else:
        return Vb_now-Vb_past

def sigma_Va(Pa_now, Pa_past, Va_now, Va_past):
    if Pa_now<Pa_past:
        return Va_now
    elif Pa_now>Pa_past:
        return 0
    else: 
        return Va_now-Va_past

In [9]:
OI = []
N = len(ask1)
for i in range(N):
    if i!=0:
        Pb_now, Pb_past, Vb_now, Vb_past = bid1[i], bid1[i-1], bid1_volumn[i], bid1_volumn[i-1]
        Pa_now, Pa_past, Va_now, Va_past = ask1[i], ask1[i-1], ask1_volumn[i], ask1_volumn[i-1]
    else:
        Pb_now, Pb_past, Vb_now, Vb_past = bid1[i], bid1[i], bid1_volumn[i], bid1_volumn[i]
        Pa_now, Pa_past, Va_now, Va_past = ask1[i], ask1[i], ask1_volumn[i], ask1_volumn[i]
    tmp = sigma_Vb(Pb_now, Pb_past, Vb_now, Vb_past)-sigma_Va(Pa_now, Pa_past, Va_now, Va_past)
    OI.append(tmp)
OI = np.array(OI)
print(OI)

[ 0.       0.00142 -0.00122 ...  0.       0.      -0.003  ]


In [10]:
df['OI'] = OI
df

,0,1,2,3,4,5,6,7,8,9,...,143,144,145,146,147,148,midprice,MPchange,y10,OI
0,0.2615,0.00353,0.2606,0.00326,0.2618,0.00200,0.2604,0.00682,0.2619,0.00164,...,0.0,2.0,2.0,2.0,2.0,2.0,0.26105,0.000364,2,0.00000
1,0.2615,0.00211,0.2606,0.00326,0.2619,0.00164,0.2604,0.00682,0.2620,0.00138,...,0.0,2.0,2.0,2.0,2.0,2.0,0.26105,0.000517,2,0.00142
2,0.2614,0.00122,0.2606,0.00326,0.2615,0.00200,0.2604,0.00682,0.2617,0.00361,...,0.0,3.0,3.0,2.0,2.0,2.0,0.26100,0.000862,2,-0.00122
3,0.2614,0.00322,0.2606,0.00326,0.2617,0.00938,0.2604,0.00682,0.2619,0.00850,...,0.0,2.0,2.0,3.0,2.0,2.0,0.26100,0.001034,2,-0.00200
4,0.2614,0.00322,0.2606,0.00326,0.2617,0.00938,0.2604,0.00682,0.2619,0.00850,...,0.0,1.0,1.0,1.0,2.0,2.0,0.26100,0.001207,2,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50945,0.1259,0.01488,0.1257,0.03166,0.1260,0.01896,0.1256,0.06681,0.1261,0.04519,...,0.0,2.0,2.0,2.0,2.0,3.0,0.12580,0.000199,2,0.00200
50946,0.1259,0.03023,0.1257,0.02166,0.1260,0.02196,0.1256,0.06681,0.1261,0.05775,...,0.0,2.0,1.0,1.0,1.0,3.0,0.12580,0.000265,2,-0.02535
50947,0.1260,0.01896,0.1257,0.02166,0.1261,0.04519,0.1256,0.06681,0.1262,0.07241,...,0.0,1.0,1.0,1.0,1.0,3.0,0.12585,0.000000,1,0.00000
50948,0.1260,0.01896,0.1257,0.02166,0.1261,0.04519,0.1256,0.06681,0.1262,0.07241,...,0.0,1.0,1.0,1.0,1.0,3.0,0.12585,0.000000,1,0.00000


## 添加OIR特征

In [11]:
def OIR(Vb,Va):
    return (Vb-Va)/(Vb+Va)

In [12]:
Va,Vb = np.array(df['1']), np.array(df['3'])

In [13]:
OIRlist = []
N = len(Va)
for i in range(N):
    Vb_now, Va_now = Vb[i], Va[i]
    tmp = OIR(Vb_now, Va_now)
    OIRlist.append(tmp)
OIRarray = np.array(OIRlist)
print(OIRarray)

[-0.03976436  0.2141527   0.45535714 ...  0.06646972  0.06646972
 -0.33333333]


In [14]:
df['OIR'] = OIRarray
df

,0,1,2,3,4,5,6,7,8,9,...,144,145,146,147,148,midprice,MPchange,y10,OI,OIR
0,0.2615,0.00353,0.2606,0.00326,0.2618,0.00200,0.2604,0.00682,0.2619,0.00164,...,2.0,2.0,2.0,2.0,2.0,0.26105,0.000364,2,0.00000,-0.039764
1,0.2615,0.00211,0.2606,0.00326,0.2619,0.00164,0.2604,0.00682,0.2620,0.00138,...,2.0,2.0,2.0,2.0,2.0,0.26105,0.000517,2,0.00142,0.214153
2,0.2614,0.00122,0.2606,0.00326,0.2615,0.00200,0.2604,0.00682,0.2617,0.00361,...,3.0,3.0,2.0,2.0,2.0,0.26100,0.000862,2,-0.00122,0.455357
3,0.2614,0.00322,0.2606,0.00326,0.2617,0.00938,0.2604,0.00682,0.2619,0.00850,...,2.0,2.0,3.0,2.0,2.0,0.26100,0.001034,2,-0.00200,0.006173
4,0.2614,0.00322,0.2606,0.00326,0.2617,0.00938,0.2604,0.00682,0.2619,0.00850,...,1.0,1.0,1.0,2.0,2.0,0.26100,0.001207,2,0.00000,0.006173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50945,0.1259,0.01488,0.1257,0.03166,0.1260,0.01896,0.1256,0.06681,0.1261,0.04519,...,2.0,2.0,2.0,2.0,3.0,0.12580,0.000199,2,0.00200,0.360550
50946,0.1259,0.03023,0.1257,0.02166,0.1260,0.02196,0.1256,0.06681,0.1261,0.05775,...,2.0,1.0,1.0,1.0,3.0,0.12580,0.000265,2,-0.02535,-0.165157
50947,0.1260,0.01896,0.1257,0.02166,0.1261,0.04519,0.1256,0.06681,0.1262,0.07241,...,1.0,1.0,1.0,1.0,3.0,0.12585,0.000000,1,0.00000,0.066470
50948,0.1260,0.01896,0.1257,0.02166,0.1261,0.04519,0.1256,0.06681,0.1262,0.07241,...,1.0,1.0,1.0,1.0,3.0,0.12585,0.000000,1,0.00000,0.066470


### 额外考虑spread

In [15]:
def spread(ask1, bid1):
    return ask1-bid1

In [16]:
ask1, bid1 = np.array(df['0']), np.array(df['2'])

In [18]:
spreadList = []
N = len(ask1)
for i in range(N):
    ask1_now, bid1_now = ask1[i], bid1[i]
    tmp = spread(ask1_now, bid1_now)
    spreadList.append(tmp)
spreadarray = np.array(spreadList)
print(spreadarray)

[0.0009 0.0009 0.0008 ... 0.0003 0.0003 0.0001]


In [19]:
df['spread'] = spreadarray
df

,0,1,2,3,4,5,6,7,8,9,...,145,146,147,148,midprice,MPchange,y10,OI,OIR,spread
0,0.2615,0.00353,0.2606,0.00326,0.2618,0.00200,0.2604,0.00682,0.2619,0.00164,...,2.0,2.0,2.0,2.0,0.26105,0.000364,2,0.00000,-0.039764,0.0009
1,0.2615,0.00211,0.2606,0.00326,0.2619,0.00164,0.2604,0.00682,0.2620,0.00138,...,2.0,2.0,2.0,2.0,0.26105,0.000517,2,0.00142,0.214153,0.0009
2,0.2614,0.00122,0.2606,0.00326,0.2615,0.00200,0.2604,0.00682,0.2617,0.00361,...,3.0,2.0,2.0,2.0,0.26100,0.000862,2,-0.00122,0.455357,0.0008
3,0.2614,0.00322,0.2606,0.00326,0.2617,0.00938,0.2604,0.00682,0.2619,0.00850,...,2.0,3.0,2.0,2.0,0.26100,0.001034,2,-0.00200,0.006173,0.0008
4,0.2614,0.00322,0.2606,0.00326,0.2617,0.00938,0.2604,0.00682,0.2619,0.00850,...,1.0,1.0,2.0,2.0,0.26100,0.001207,2,0.00000,0.006173,0.0008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50945,0.1259,0.01488,0.1257,0.03166,0.1260,0.01896,0.1256,0.06681,0.1261,0.04519,...,2.0,2.0,2.0,3.0,0.12580,0.000199,2,0.00200,0.360550,0.0002
50946,0.1259,0.03023,0.1257,0.02166,0.1260,0.02196,0.1256,0.06681,0.1261,0.05775,...,1.0,1.0,1.0,3.0,0.12580,0.000265,2,-0.02535,-0.165157,0.0002
50947,0.1260,0.01896,0.1257,0.02166,0.1261,0.04519,0.1256,0.06681,0.1262,0.07241,...,1.0,1.0,1.0,3.0,0.12585,0.000000,1,0.00000,0.066470,0.0003
50948,0.1260,0.01896,0.1257,0.02166,0.1261,0.04519,0.1256,0.06681,0.1262,0.07241,...,1.0,1.0,1.0,3.0,0.12585,0.000000,1,0.00000,0.066470,0.0003


###  OI和OIR都经过spread修正

In [23]:
OI_array,OIR_array,spread_array = np.array(df['OI']), np.array(df['OIR']), np.array(df['spread'])


In [24]:
OI_std = OI_array/spread_array
OIR_std = OIR_array/spread_array

In [28]:
# Creating the data frame
details = {
    'OI_std': OI_std,
    'OIR_std': OIR_std
}
# creating a Dataframe object
new_df = pd.DataFrame(details)
  
# define standard scaler
scaler = StandardScaler()
  
# transform data
new_df = scaler.fit_transform(new_df)
new_df = pd.DataFrame(new_df)
new_df

,0,1
0,0.008952,0.021980
1,0.036117,0.111601
2,-0.017305,0.216825
3,-0.034091,0.038466
4,0.008952,0.038466
...,...,...
50945,0.181124,0.608675
50946,-2.173335,-0.226303
50947,0.008952,0.106397
50948,0.008952,0.106397


In [29]:
col = ['OI_std', 'OIR_std']
new_df.columns = col
new_df

,OI_std,OIR_std
0,0.008952,0.021980
1,0.036117,0.111601
2,-0.017305,0.216825
3,-0.034091,0.038466
4,0.008952,0.038466
...,...,...
50945,0.181124,0.608675
50946,-2.173335,-0.226303
50947,0.008952,0.106397
50948,0.008952,0.106397


In [30]:
df

,0,1,2,3,4,5,6,7,8,9,...,145,146,147,148,midprice,MPchange,y10,OI,OIR,spread
0,0.2615,0.00353,0.2606,0.00326,0.2618,0.00200,0.2604,0.00682,0.2619,0.00164,...,2.0,2.0,2.0,2.0,0.26105,0.000364,2,0.00000,-0.039764,0.0009
1,0.2615,0.00211,0.2606,0.00326,0.2619,0.00164,0.2604,0.00682,0.2620,0.00138,...,2.0,2.0,2.0,2.0,0.26105,0.000517,2,0.00142,0.214153,0.0009
2,0.2614,0.00122,0.2606,0.00326,0.2615,0.00200,0.2604,0.00682,0.2617,0.00361,...,3.0,2.0,2.0,2.0,0.26100,0.000862,2,-0.00122,0.455357,0.0008
3,0.2614,0.00322,0.2606,0.00326,0.2617,0.00938,0.2604,0.00682,0.2619,0.00850,...,2.0,3.0,2.0,2.0,0.26100,0.001034,2,-0.00200,0.006173,0.0008
4,0.2614,0.00322,0.2606,0.00326,0.2617,0.00938,0.2604,0.00682,0.2619,0.00850,...,1.0,1.0,2.0,2.0,0.26100,0.001207,2,0.00000,0.006173,0.0008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50945,0.1259,0.01488,0.1257,0.03166,0.1260,0.01896,0.1256,0.06681,0.1261,0.04519,...,2.0,2.0,2.0,3.0,0.12580,0.000199,2,0.00200,0.360550,0.0002
50946,0.1259,0.03023,0.1257,0.02166,0.1260,0.02196,0.1256,0.06681,0.1261,0.05775,...,1.0,1.0,1.0,3.0,0.12580,0.000265,2,-0.02535,-0.165157,0.0002
50947,0.1260,0.01896,0.1257,0.02166,0.1261,0.04519,0.1256,0.06681,0.1262,0.07241,...,1.0,1.0,1.0,3.0,0.12585,0.000000,1,0.00000,0.066470,0.0003
50948,0.1260,0.01896,0.1257,0.02166,0.1261,0.04519,0.1256,0.06681,0.1262,0.07241,...,1.0,1.0,1.0,3.0,0.12585,0.000000,1,0.00000,0.066470,0.0003


### 拼接

In [37]:
df[['OIstd','OIRstd']] = new_df

In [38]:
df

,0,1,2,3,4,5,6,7,8,9,...,147,148,midprice,MPchange,y10,OI,OIR,spread,OIstd,OIRstd
0,0.2615,0.00353,0.2606,0.00326,0.2618,0.00200,0.2604,0.00682,0.2619,0.00164,...,2.0,2.0,0.26105,0.000364,2,0.00000,-0.039764,0.0009,0.008952,0.021980
1,0.2615,0.00211,0.2606,0.00326,0.2619,0.00164,0.2604,0.00682,0.2620,0.00138,...,2.0,2.0,0.26105,0.000517,2,0.00142,0.214153,0.0009,0.036117,0.111601
2,0.2614,0.00122,0.2606,0.00326,0.2615,0.00200,0.2604,0.00682,0.2617,0.00361,...,2.0,2.0,0.26100,0.000862,2,-0.00122,0.455357,0.0008,-0.017305,0.216825
3,0.2614,0.00322,0.2606,0.00326,0.2617,0.00938,0.2604,0.00682,0.2619,0.00850,...,2.0,2.0,0.26100,0.001034,2,-0.00200,0.006173,0.0008,-0.034091,0.038466
4,0.2614,0.00322,0.2606,0.00326,0.2617,0.00938,0.2604,0.00682,0.2619,0.00850,...,2.0,2.0,0.26100,0.001207,2,0.00000,0.006173,0.0008,0.008952,0.038466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50945,0.1259,0.01488,0.1257,0.03166,0.1260,0.01896,0.1256,0.06681,0.1261,0.04519,...,2.0,3.0,0.12580,0.000199,2,0.00200,0.360550,0.0002,0.181124,0.608675
50946,0.1259,0.03023,0.1257,0.02166,0.1260,0.02196,0.1256,0.06681,0.1261,0.05775,...,1.0,3.0,0.12580,0.000265,2,-0.02535,-0.165157,0.0002,-2.173335,-0.226303
50947,0.1260,0.01896,0.1257,0.02166,0.1261,0.04519,0.1256,0.06681,0.1262,0.07241,...,1.0,3.0,0.12585,0.000000,1,0.00000,0.066470,0.0003,0.008952,0.106397
50948,0.1260,0.01896,0.1257,0.02166,0.1261,0.04519,0.1256,0.06681,0.1262,0.07241,...,1.0,3.0,0.12585,0.000000,1,0.00000,0.066470,0.0003,0.008952,0.106397


## 将上面的方法封装成函数

In [40]:
def sigma_Vb(Pb_now, Pb_past, Vb_now, Vb_past):
    if Pb_now<Pb_past:
        return 0
    elif Pb_now>Pb_past:
        return Vb_now
    else:
        return Vb_now-Vb_past

def sigma_Va(Pa_now, Pa_past, Va_now, Va_past):
    if Pa_now<Pa_past:
        return Va_now
    elif Pa_now>Pa_past:
        return 0
    else: 
        return Va_now-Va_past

def OIR(Vb,Va):
    return (Vb-Va)/(Vb+Va)

def spread(ask1, bid1):
    return ask1-bid1

In [39]:
def moreFeature(inpath,outpath):
    df = pd.read_csv(inpath)
    # 添加OI
    ask1,ask1_volumn,bid1,bid1_volumn = np.array(df['0']),np.array(df['1']),np.array(df['2']),np.array(df['3'])
    OI = []
    N = len(ask1)
    for i in range(N):
        if i!=0:
            Pb_now, Pb_past, Vb_now, Vb_past = bid1[i], bid1[i-1], bid1_volumn[i], bid1_volumn[i-1]
            Pa_now, Pa_past, Va_now, Va_past = ask1[i], ask1[i-1], ask1_volumn[i], ask1_volumn[i-1]
        else:
            Pb_now, Pb_past, Vb_now, Vb_past = bid1[i], bid1[i], bid1_volumn[i], bid1_volumn[i]
            Pa_now, Pa_past, Va_now, Va_past = ask1[i], ask1[i], ask1_volumn[i], ask1_volumn[i]
        tmp = sigma_Vb(Pb_now, Pb_past, Vb_now, Vb_past)-sigma_Va(Pa_now, Pa_past, Va_now, Va_past)
        OI.append(tmp)
    OI = np.array(OI)
    df['OI'] = OI
    # 添加OIR
    Va,Vb = np.array(df['1']), np.array(df['3'])
    OIRlist = []
    N = len(Va)
    for i in range(N):
        Vb_now, Va_now = Vb[i], Va[i]
        tmp = OIR(Vb_now, Va_now)
        OIRlist.append(tmp)
    OIRarray = np.array(OIRlist)
    df['OIR'] = OIRarray
    # 添加spread
    ask1, bid1 = np.array(df['0']), np.array(df['2'])
    spreadList = []
    N = len(ask1)
    for i in range(N):
        ask1_now, bid1_now = ask1[i], bid1[i]
        tmp = spread(ask1_now, bid1_now)
        spreadList.append(tmp)
    spreadarray = np.array(spreadList)
    df['spread'] = spreadarray
    # 添加OIstd和OIRstd
    OI_array,OIR_array,spread_array = np.array(df['OI']), np.array(df['OIR']), np.array(df['spread'])
    OI_std = OI_array/spread_array
    OIR_std = OIR_array/spread_array
    # Creating the data frame
    details = {
        'OI_std': OI_std,
        'OIR_std': OIR_std
    }
    # creating a Dataframe object
    new_df = pd.DataFrame(details)
    
    # define standard scaler
    scaler = StandardScaler()
    
    # transform data
    new_df = scaler.fit_transform(new_df)
    new_df = pd.DataFrame(new_df)
    df[['OIstd','OIRstd']] = new_df
    df.to_csv(outpath, index=False)

In [81]:
inpath = '../../data/random_processed/stock0/newY_train7_part0.csv'
outpath = '../../data/random_processed/stock0/moreX_train7_part0.csv'
moreFeature(inpath, outpath)

In [82]:
df = pd.read_csv('../../data/random_processed/stock0/moreX_train7_part0.csv')
df

,0,1,2,3,4,5,6,7,8,9,...,142,143,midprice,MPchange,y10,OI,OIR,spread,OIstd,OIRstd
0,0.3466,0.00100,0.3455,0.00369,0.3467,0.00702,0.3450,0.00200,0.3468,0.00200,...,0.0,0.0,0.34605,-0.468747,0,0.00000,0.573561,0.0011,0.162970,0.173720
1,0.1730,0.06079,0.1727,0.03289,0.1731,0.04800,0.1726,0.02355,0.1732,0.05670,...,0.0,0.0,0.17285,0.061788,1,-0.06079,-0.297822,0.0003,-1.606010,-0.324798
2,0.1261,0.01511,0.1260,0.02800,0.1262,0.01216,0.1259,0.04169,0.1263,0.02200,...,0.0,0.0,0.12605,0.531813,2,-0.01511,0.299003,0.0001,-1.156126,0.986476
3,0.1304,0.02000,0.1302,0.01092,0.1305,0.03820,0.1301,0.05760,0.1306,0.01300,...,0.0,0.0,0.13030,0.482310,1,0.01092,-0.293661,0.0002,0.639626,-0.481371
4,0.1264,0.00318,0.1262,0.01770,0.1265,0.00500,0.1261,0.01650,0.1266,0.03910,...,0.0,0.0,0.12630,0.703919,2,-0.00318,0.695402,0.0002,0.024164,1.146810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50945,0.3503,0.01471,0.3500,0.00171,0.3505,0.00100,0.3495,0.01657,0.3506,0.00450,...,0.0,0.0,0.35015,-0.214015,1,0.00171,-0.791717,0.0003,0.212731,-0.866826
50946,0.2681,0.00100,0.2673,0.00076,0.2683,0.00388,0.2672,0.01136,0.2684,0.00267,...,0.0,0.0,0.26770,-0.065247,1,-0.00100,-0.136364,0.0008,0.152058,-0.054071
50947,0.3524,0.00300,0.3515,0.00200,0.3525,0.00293,0.3514,0.00690,0.3529,0.00490,...,0.0,0.0,0.35195,-0.313823,0,0.00200,-0.200000,0.0009,0.182370,-0.071114
50948,0.1311,0.00288,0.1310,0.01471,0.1312,0.03018,0.1309,0.01768,0.1313,0.01500,...,0.0,0.0,0.13105,0.000000,1,-0.00288,0.672541,0.0001,-0.088453,2.216303
